In [ ]:
# Package dependencies
import numpy as np
import pandas as pd
import seaborn as sns
from statannot import add_stat_annotation
import statsmodels
from matplotlib import pyplot as plt
from glob import glob
import os.path
from os.path import exists
import subprocess
from os import path
import mygene
import scipy
import pydove as dv
import sqlite3
import gseapy as gp
import math
import sklearn
import re
from gseapy.plot import gseaplot, dotplot, barplot
from scipy.stats import pearsonr
plt.rcParams["figure.figsize"] = (5,4)
import matplotlib
matplotlib.rcParams["figure.dpi"] = 200
import pygame
blue = np.array(pygame.Color("#2D77B2"))/255
red = np.array(pygame.Color("#CB2F45"))/255
data_dir = "/clusterfs/nilah/rkchung/data/expred/"
gtex_dir = "/clusterfs/genomicdata/GTEx/eQTL_files/"
colors = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/pal_tissue.csv")
medawar = ['Artery_Tibial', 'Esophagus_Gastroesophageal_Junction', 'Esophagus_Muscularis', 'Colon_Sigmoid', 'Artery_Aorta', 'Artery_Coronary', 'Muscle_Skeletal', 'Esophagus_Mucosa', 'Testis', 'Adipose_Visceral_Omentum', 'Thyroid', 'Cells_Cultured_fibroblasts', 'Skin_Sun_Exposed_Lower_leg', 'Pancreas', 'Liver', 'Stomach', 'Heart_Left_Ventricle', 'Adipose_Subcutaneous']
notmedawar = ['Prostate', 'Colon_Transverse', 'Breast_Mammary_Tissue', 'Whole_Blood', 'Lung']

In [ ]:
# Generate mapping of ensembl id -> gene name
tissues = glob(gtex_dir+"GTEx_Analysis_v8_eQTL_expression_matrices/*.bed.gz")

genes = []
for i in range(len(tissues)):
    genes += list(pd.read_csv(tissues[i], usecols = ['gene_id'], sep="\t")["gene_id"]) # list of gtex genes - ensembl id format
genes = list(set(genes))
print("Found %s genes" % len(genes))
mg = mygene.MyGeneInfo()
ens = [g.split(".")[0] for g in genes]
ginfo = mg.querymany(ens, scopes='ensembl.gene', returnall=True)["out"]
print(len(ginfo))
counter = 0
#comm_genes = []
mapping = {} # ensg id to common name
for j in range(len(ginfo)): 
    if "symbol" in ginfo[j]:
        mapping[ginfo[j]["query"]] = ginfo[j]["symbol"]
print("%s Unmapped genes" % (len(genes)-len(mapping)))

# Make table of all info: age/genetic performance, cohort performance
(Need to run multiSNP age model for each tissue first (Snakemake --jobs 20))

In [ ]:
tissues = glob(gtex_dir+"GTEx_Analysis_v8_eQTL_expression_matrices/*.bed.gz")
# Get constraint measurs from gnomad (https://gnomad.broadinstitute.org/downloads#v2-constraint)
plidf = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/gnomad.v2.1.1.lof_metrics.by_gene.txt", sep="\t")
all_df = []

for i in range(len(tissues)):
    print(int(subprocess.check_output("zcat %s | awk '{print NF; exit}'" % tissues[i], shell=True)))
    tiss = tissues[i].split("expression_matrices/")[1].split(".v8.normalized_expression")[0]

    if int(subprocess.check_output("zcat %s | awk '{print NF; exit}'" % tissues[i], shell=True)) > 200 and exists(data_dir+"output/res_%s_agereg_proc0.npy" % tiss) and exists(data_dir+"output/res_%s_cohort_proc.npy" % tiss): # checks if num of individuals is enough
        output = np.load(data_dir+"output/res_%s_agereg_proccorr.npy"  % tiss, allow_pickle=True).item()
        #outputcohort = np.load(data_dir+"output/res_%s_cohort_proc.npy"  % tiss, allow_pickle=True).item()
        print("Loaded res %s" % tiss)

        expr = pd.read_csv(gtex_dir+"GTEx_Analysis_v8_eQTL_expression_matrices/%s.v8.normalized_expression.bed.gz" % tiss, sep="\t")
        genes = expr["gene_id"].to_numpy()

        gene_r2 = output["gene_r2"]
        age_r2 = output["age_r2"]
        age_coef = np.array([c[-1] for c in output["coef"]])
        env_r2 = np.minimum(np.ones(len(gene_r2)) - gene_r2 - age_r2, 1)
        #old_r2 = np.array(outputcohort["old_t_r2"]).reshape(-1)
        #young_r2 = np.array(outputcohort["young_t_r2"]).reshape(-1)
        print(len(old_r2))
        print(len(age_r2))
        
        df = pd.DataFrame(np.array([genes[:len(gene_r2)], np.array(gene_r2).reshape(-1), np.array(age_r2).reshape(-1), 
                                    np.array(env_r2).reshape(-1), age_coef]).transpose(), 
                          columns=["Gene", "Genetic R^2", "Age R^2", "Env R^2", "Age Coef"])

        df["gene"] = df["Gene"].apply(lambda x: mapping[x.split(".")[0]] if x.split(".")[0] in mapping else np.nan)
        df["Tissue"] = tiss
        df = df[["Tissue", "Gene", "gene", "Genetic R^2", "Age R^2", "Env R^2", "Age Coef"]]
        #df = df[["Tissue", "Gene", "gene", "Genetic R^2", "Age R^2", "Env R^2", "Age Coef", "Young R^2", "Old R^2"]]
        df = df.merge(plidf[["gene", "pLI"]], on="gene", how="left")
        df["Genetic R^2"] = df["Genetic R^2"].astype(float)
        df["Age R^2"] = df["Age R^2"].astype(float)
        df["Age Coef"] = df["Age Coef"].astype(float)
        df = df.drop_duplicates(subset=["Gene"])
        print(df)
        all_df += [df]
all_df = pd.concat(all_df)
print(all_df)
all_df.to_csv("/clusterfs/nilah/rkchung/data/expred/all_res3.csv", index=False)

# Plot age coefficient for particular gene

In [ ]:
#tiss = "Whole_Blood"
tiss = "Esophagus_Mucosa"
#tiss = "Prostate"
#tiss = "Adipose_Visceral_Omentum"
expr = pd.read_csv(gtex_dir+"GTEx_Analysis_v8_eQTL_expression_matrices/%s.v8.normalized_expression.bed.gz" % tiss, sep="\t")
age_df = pd.read_csv(data_dir+"phs000424.v8.pht002743.v8.p2.c1.GTEx_Sample_Info.txt", sep="\t")[["SUBJID", "AGE"]]
age_df = age_df.drop_duplicates()
indiv = list(expr.columns)[4:]
age_df["norm_age"] = (age_df["AGE"]-age_df["AGE"].mean())/age_df["AGE"].std() 
indiv_age_map = dict(zip(age_df.SUBJID, age_df.norm_age))
indiv_ages = [indiv_age_map[i] for i in indiv]

In [ ]:
#common_gene = "CDCA7L" # blood gene
all_data = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/all_res2.csv")
#common_gene = "LETM1"
#common_gene = "CDKN2A"
common_gene = "MCMBP"
#common_gene = "RARRES2"
#common_gene = "CDK11B"
#common_gene = "EIF3J"
gene_id = list(all_data[all_data["gene"]==common_gene]["Gene"])[0]
expr2 = expr[expr["gene_id"]==gene_id]

# Remove covariates
from sklearn.linear_model import LinearRegression 
adj_dir = "/global/scratch-old/ryo10244201/analysis/Predixcan_cov/" 
covar = pd.read_csv(adj_dir + tiss + "_covariates.csv", index_col=0)  

covar.index = pd.Index(covar["SUBJID"])
#print(covar)
covar.drop(["AGE", "SUBJID"], axis=1, inplace=True)
covar = covar.T
skclf = LinearRegression()
skclf.fit(np.transpose(covar[indiv].to_numpy()), np.transpose(expr2[indiv].to_numpy()))
covar_r2 = skclf.score(np.transpose(covar[indiv].to_numpy()), np.transpose(expr2[indiv].to_numpy()))
expr2[indiv] = expr2[indiv] - skclf.predict(np.transpose(covar[indiv].to_numpy())).reshape(1, -1)
print(skclf.coef_)
print("Covar R^2 %s" % covar_r2)

In [ ]:
all_data[(all_data["gene"]==common_gene)&(all_data["Tissue"]==tiss)]

In [ ]:
plt.figure(dpi=400)
expression = list(expr2[expr2["gene_id"]==gene_id].to_numpy().reshape(-1)[4:].astype(float))
plt.scatter(indiv_ages, expression, alpha=0.5, color="black")
endpoints = [min(indiv_ages), max(indiv_ages)]
coef = list(all_data[(all_data["gene"]==common_gene)&(all_data["Tissue"]==tiss)]["Age Coef"].astype(float))[0]
print(all_data[(all_data["gene"]==common_gene)&(all_data["Tissue"]==tiss)]["Age Coef"])
yend = np.array(endpoints)*coef+np.mean(expression)
plt.plot(endpoints, yend, c="black", linewidth=2.5)
plt.text(np.mean(endpoints), max(expression)-0.05, r'$\beta_{Age}$: %.4f' % (coef), ha="center", size=12)

plt.xlabel("Normalized Age")
plt.ylabel("Normalized Expression")
plt.title("Expression vs Age for %s in %s" % (common_gene, "Esophagus_Muco"))
plt.show()
plt.clf()
print(coef)

clf = sklearn.linear_model.LinearRegression()
clf.fit(np.array(indiv_ages).reshape(-1,1), expression)
print(clf.coef_)

# Consistency of heritability measures with predixcan

In [ ]:
# Generate figure with heritability correlation to predixcan models
tissues = glob(gtex_dir+"GTEx_Analysis_v8_eQTL_expression_matrices/*.bed.gz")
all_data = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/all_res2.csv")
print(all_data)
pearsonrs = []
measured_tiss = []
for i in range(len(tissues)):
    print(int(subprocess.check_output("zcat %s | awk '{print NF; exit}'" % tissues[i], shell=True)))
    tiss = tissues[i].split("expression_matrices/")[1].split(".v8.normalized_expression")[0]
    if int(subprocess.check_output("zcat %s | awk '{print NF; exit}'" % tissues[i], shell=True)) > 200 and exists(data_dir+"output/res_%s_cohort_proc.npy" % tiss): # checks if num of individuals is enough
        print(tiss)
        df = all_data[all_data["Tissue"]==tiss]
        connection = sqlite3.connect("/clusterfs/nilah/rkchung/data/predixcan/elastic_net_models/en_%s.db" % tiss)
        cursor = connection.cursor()
        cursor.execute('SELECT gene from weights')
        result = cursor.fetchall()
        predix_genes = set([r[0].split(".")[0] for r in result])
        cursor.execute('SELECT gene, cv_R2_avg from extra')
        result = cursor.fetchall()
        predix_df = pd.DataFrame(result, columns=["Gene", "Predix R^2"])
        merged = predix_df.merge(df, on="Gene")
        pearsonrs += [pearsonr(merged["Predix R^2"], np.array(merged["Genetic R^2"]).reshape(-1))[0]]
        measured_tiss += [tiss]
        print(pearsonrs[-1])
print("Average Pearson R of heritability: %s" % np.mean(pearsonrs))

In [ ]:
# Plot scatterplot of heritabilities for predixcan and our model for single tissue 
merged = predix_df.merge(df, on="Gene")
plt.scatter(merged["Predix R^2"], merged["Genetic R^2"], s=1)
plt.xlabel(r'PrediXcan $R^2$')
plt.ylabel(r'Our Model Genetic $R^2$')
plt.title(r'Consistency of $h^2$ measures for %s' % tiss)
print(pearsonr(merged["Predix R^2"], np.array(merged["Genetic R^2"]).reshape(-1)))
plt.show()

In [ ]:
plot_df = pd.DataFrame(np.array([measured_tiss, pearsonrs]).T, columns=["Tissue", "Pearson's r of expression heritability PrediXcan vs AGE model"])
plot_df["Pearson's r of expression heritability PrediXcan vs AGE model"] = plot_df["Pearson's r of expression heritability PrediXcan vs AGE model"].astype(float)
plot_df = plot_df.merge(colors, on="Tissue")
plot_df["Tissue"] = plot_df["Tissue"].apply(lambda x: re.sub('[_]', ' ', x))
plot_df = plot_df.sort_values("Pearson's r of expression heritability PrediXcan vs AGE model")
ax=sns.barplot(data=plot_df, x="Pearson's r of expression heritability PrediXcan vs AGE model", y="Tissue", color="grey")


# pLI vs $\beta_{age}$

In [ ]:
# Plot pLI vs age coef
tissues = glob(gtex_dir+"GTEx_Analysis_v8_eQTL_expression_matrices/*.bed.gz")
from matplotlib.lines import Line2D
#short_list = ["Colon_Sigmoid", "Lung"]
#tissues = [t for t in tissues if np.any([s in t for s in short_list])]
#plidf = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/exac_data_pLI.txt", sep="\t")
plidf = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/gnomad_constraint_metrics.tsv", sep="\t")
#plidf = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/gnomad.v2.1.1.lof_metrics.by_gene.txt", sep="\t")
meta_res = []
cons_metric = "pLI"
quad = False # show quadrant colors and proportions
driving = False
violin = False # violinplots
quadrants = []
all_data = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/all_res2.csv")
for i in range(len(tissues)):
    print(int(subprocess.check_output("zcat %s | awk '{print NF; exit}'" % tissues[i], shell=True)))
    tiss = tissues[i].split("expression_matrices/")[1].split(".v8.normalized_expression")[0]

    if int(subprocess.check_output("zcat %s | awk '{print NF; exit}'" % tissues[i], shell=True)) > 200 and exists(data_dir+"output/res_%s_agereg_proc0.npy" % tiss): # checks if num of individuals is enough
        #output = np.load(data_dir+"output/res_%s_agereg_proccorr.npy"  % tiss, allow_pickle=True).item()
        df = all_data[all_data["Tissue"]==tiss]
        print("Loaded res %s" % tiss)
        subdf = df[df["Age R^2"]>0]
        subdf = subdf.drop_duplicates(subset=["Gene"])
        subdf["Age Coef"] = subdf["Age Coef"].astype(float)
        subdf["|Age Coef|"] = np.abs(subdf["Age Coef"])
        subdf = subdf.dropna()
    
        #thresh = 0.1
        #subdf["|Age R^2| > %s" % thresh] = subdf["|Young-Old R^2|"] > thresh

        import matplotlib
        matplotlib.rcParams["figure.dpi"] = 400
        fig, ax = plt.subplots()
        # additional color to visualization to show pli/age coef quartiles
        # add percentage of data at above/below threshold
        #colors = ["blue", "cyan", "red", "magenta"]
        if quad:
            subdf["quad"] = 2*(subdf[cons_metric]>0.5).astype(int) + (subdf["Age Coef"]>0).astype(int)
            subdf["quadcolor"] = subdf["quad"].apply(lambda q: colors[q])
            res = dv.regplot(data=subdf, y="Age Coef", x=cons_metric, order=1, ax=ax,
                             scatter_kws={'s':3, 'alpha':0.2, 'c':subdf["quadcolor"]}, line_kws={'c':"black", 'linewidth':3})
            plt.text(0.8, max(list(subdf["Age Coef"]))-0.25, 
                     '%.1f%%' % (100*np.mean(subdf["quad"]==3)), 
                     size=12, ha="center")
            plt.text(0.8, min(list(subdf["Age Coef"]))+0.15, 
                     '%.1f%%' % (100*np.mean(subdf["quad"]==2)), 
                     size=12, ha="center")
            plt.text(0.2, max(list(subdf["Age Coef"]))-0.25, 
                     '%.1f%%' % (100*np.mean(subdf["quad"]==1)), 
                     size=12, ha="center")
            plt.text(0.2, min(list(subdf["Age Coef"]))+0.15, 
                     '%.1f%%' % (100*np.mean(subdf["quad"]==0)), 
                     size=12, ha="center")
            quadrants += [[tiss, 100*np.mean(subdf["quad"]==0), 100*np.mean(subdf["quad"]==1), 
                           100*np.mean(subdf["quad"]==2), 100*np.mean(subdf["quad"]==3)]]
        elif driving:
            #colors = ["orange", "black"]
            subdf["driving_colors"] = list(subdf["gene"].apply(lambda g: colors[int(g in driving_set)]))
            subdf = subdf.sort_values("driving_colors", ascending=False)
            res = dv.regplot(data=subdf, y="Age Coef", x=cons_metric, order=1, ax=ax,
                             scatter_kws={'s':3, 'alpha':0.3, 'c':subdf["driving_colors"]}, line_kws={'c':"black", 'linewidth':0})
        else:
            res = dv.regplot(data=subdf, y="Age Coef", x=cons_metric, order=1, ax=ax,
                             scatter_kws={'s':3, 'alpha':0.3, 'c':blue}, line_kws={'c':"black", 'linewidth':3})

        plt.text(0.5, max(list(subdf["Age Coef"]))-0.05, 
                     'Fit slope: %.4f; P-value: %.1e' % (res.params[1], res.pvalues[1]) 
                     if res.pvalues[1]!=0 else 'fit slope: %.4f; P-value: 0' % (res.params[1]), 
                     size=14, ha="center")
        if cons_metric == "oe_lof":
            plt.xlim(0, 2)
        plt.xlabel("Gene Constraint (%s)" % cons_metric, fontsize=16)
        plt.ylabel(r'Age of Expression ($\beta_{Age})$', fontsize=16)
        plt.title(r'%s vs $\beta_{Age}$ for %s' % (cons_metric, re.sub('[_]', ' ', tiss)), fontsize=18)
        #plt.ylabel(r'$\beta_{Age}$', fontsize=16)
        #plt.title(r'%s vs $\beta_{Age}$ for %s' % (cons_metric, tiss), fontsize=18)
        plt.savefig("%s_medawardriver" % tiss)
        plt.show()
        plt.clf()
        thresh = 0.5

        if violin:
            y = "Age Coef"
            x = cons_metric + " > %s" % thresh
            #subdf["|Young-Old R^2| > %s" % thresh] = subdf["|Young-Old R^2| > %s" % thresh].astype(str)
            subdf[cons_metric + " > %s" % thresh] = (subdf[cons_metric]>thresh).astype(str)
            ax = sns.violinplot(x=x, y=y, data=subdf, order=["True","False"])
            test_results = add_stat_annotation(ax, data=subdf, x=x, y=y,
                                               test='Mann-Whitney', text_format='full', loc='inside',
                                               box_pairs=[["True","False"]]) 
            Means = subdf.groupby(x)[y].mean()
            print(Means)
            plt.scatter(x=range(len(Means)),y=[Means["True"], Means["False"]],c="r")
            plt.title("%s for %s" % (y, tiss))
            plt.figure(dpi=400)
            plt.show()
            plt.clf()

            ax = sns.violinplot(x=x, y=y, data=subdf, order=["True","False"])
            plt.title("%s for %s; %s, %s" % (y, tiss, len(subdf[subdf[x]=="True"]), len(subdf[subdf[x]=="False"])))
            Q1 = np.percentile(sorted(list(subdf[y])), 25, interpolation = 'midpoint')
            Q80 = np.percentile(sorted(list(subdf[y])), 80, interpolation = 'midpoint')
            Means = subdf.groupby(x)[y].mean()
            plt.scatter(x=range(len(Means)),y=[Means["True"], Means["False"]],c="r")
            mean_diff = Means["True"] - Means["False"]
            ax.set_ylim([Q1, Q80])
            #ax.set(ylim=(0, 100000))
            plt.show()
            plt.clf()
        if quad:
            meta_res += [[tiss, res.params[1], res.pvalues[1], len(subdf), list(subdf["Age Coef"]), 100*np.mean(subdf["quad"]==0), 100*np.mean(subdf["quad"]==1), 100*np.mean(subdf["quad"]==2)]]
        else:
            meta_res += [[tiss, res.params[1], res.pvalues[1], len(subdf)]]
        #print(meta_res[-1])

In [ ]:
# Plot age coef vs pli slope for each tissue
meta_res_dict = {r[0]:[r[1:]] for r in meta_res}
print(meta_res_dict)
df = pd.DataFrame(meta_res, columns=["Tissue", "Slope of Age Coef vs pLI", "P-value", "Num Genes"])
df[r'Slope of pLI vs $\beta_{Age}$'] = df["Slope of Age Coef vs pLI"].astype(float)
df["P-value"] = df["P-value"].astype(float)
df = df.sort_values(r'Slope of pLI vs $\beta_{Age}$')
print(df)
df = df.merge(colors, on="Tissue")
df["Tissue"] = df["Tissue"].apply(lambda t: re.sub('[_]', ' ', t))
ax=sns.barplot(data=df, x=r'Slope of pLI vs $\beta_{Age}$', y="Tissue", palette=list(df["pal.tissue"]))

def pval_asterisk(pvalue):
    if pvalue <= 0.0001:
        return "****"
    elif pvalue <= 0.001:
        return "***"
    elif pvalue <= 0.01:
        return "**"
    elif pvalue <= 0.05:
        return "*"
    return "ns"

plt.yticks(fontsize=8)
x_position = df[r'Slope of pLI vs $\beta_{Age}$'] + np.sign(df[r'Slope of pLI vs $\beta_{Age}$'])*0.006
med = []
notmed = []
for idx, pval in enumerate(df["P-value"].apply(pval_asterisk)):
    plt.text(x=x_position.iloc[idx], y=(idx+0.4 if pval=="ns" else idx+0.6), s=pval, ha="center")
    if pval == "***" or pval == "****":
        if list(df[r'Slope of pLI vs $\beta_{Age}$'])[idx] < 0:
            med += [list(df["Tissue"])[idx]]
        else:
            notmed += [list(df["Tissue"])[idx]]
print(med)
print(notmed)
plt.text(x=0.007, y=0, s="*: P-value<0.05\n**: P-value<0.01\n***: P-value<0.001\n****: P-value<0.0001", va="top")
plt.title(r'Relationship between pLI and $\beta_{Age}$')
plt.xlim(np.min(df[r'Slope of pLI vs $\beta_{Age}$'])-0.012, np.max(df[r'Slope of pLI vs $\beta_{Age}$'])+0.012)
plt.show()
plt.clf()

# Quadrant analysis (pli vs $\beta_{age}$)

In [ ]:
#pathway_df = pd.DataFrame(np.array(pathway_fracs), columns=["Proportion of pathways", "Tissue type"])
all_data = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/all_res2.csv")
all_data = all_df[all_df["Age R^2"].astype(float) > 0]
all_data["quadrant"] = 2*(all_data["pLI"]>0.5)+(all_data["Age Coef"]>0)
#pathway_df["Proportion of pathways"] = pathway_df["Proportion of pathways"].astype(float)
from collections import Counter
quad_df = all_data.groupby("Tissue")["quadrant"].apply(lambda qs: np.array([Counter(qs)[i] for i in range(4)])/len(qs))
quad_df = pd.DataFrame(quad_df.to_list(), columns=[r'pLI<0.5, $\beta_{age}<0$',r'pLI<0.5, $\beta_{age}>0$',r'pLI>0.5, $\beta_{age}<0$',r'pLI>0.5, $\beta_{age}>0$'], index=quad_df.index)
#quad_df = quad_df.set_index("Tissue")
quad_df = quad_df.stack()
quad_df = quad_df.reset_index()

quad_df = quad_df.rename(columns={"level_1":"Quadrant", 0:"Proportion of genes"})

quad_df["pLI>0.5"] = quad_df["Quadrant"].apply(lambda q: "pLI>0.5" in q).astype(str)
quad_df["Tissue type"] = quad_df["Tissue"].astype(str).apply(lambda t: "Medawar" if t in medawar else ("Not Medawar" if t in notmedawar else np.nan))
quad_df = quad_df.dropna()
print(quad_df)
print(list(quad_df["Tissue type"]))

box_pairs = [((r'pLI<0.5, $\beta_{age}<0$', "Medawar"), (r'pLI<0.5, $\beta_{age}<0$', "Not Medawar")), ((r'pLI<0.5, $\beta_{age}>0$', "Medawar"), (r'pLI<0.5, $\beta_{age}>0$', "Not Medawar")), ((r'pLI>0.5, $\beta_{age}<0$', "Medawar"), (r'pLI>0.5, $\beta_{age}<0$', "Not Medawar")), ((r'pLI>0.5, $\beta_{age}>0$', "Medawar"), (r'pLI>0.5, $\beta_{age}>0$', "Not Medawar"))]
ax = sns.boxplot(data=quad_df, x="Quadrant", y="Proportion of genes", hue="Tissue type")
test_results = add_stat_annotation(ax, data=quad_df, x="Quadrant", y="Proportion of genes",
                                   hue="Tissue type", test='Mann-Whitney', loc='inside', verbose=2,
                                   box_pairs=box_pairs)
plt.xticks(rotation = 30)
plt.title("Proportion of genes in each quadrant")

# Hallmark pathway analysis

In [ ]:
# Requires download of MSigDB hallmark pathways from  (http://www.gsea-msigdb.org/gsea/msigdb/collections.jsp#H)
with open(data_dir+'h.all.v7.4.symbols.gmt') as f:
    lines = f.readlines()
pathways = {l.split("\t")[0]:l.strip().split("\t")[2:] for l in lines}
print(pathways.keys())
print(pathways["HALLMARK_TGF_BETA_SIGNALING"])

import pygame
blue = np.array(pygame.Color("#2D77B2"))/255
red = np.array(pygame.Color("#CB2F45"))/255

all_df = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/all_res.csv")
all_df = all_df[all_df["Age R^2"]>0]
all_df = all_df.dropna()
all_df = all_df[all_df["Tissue"].apply(lambda t: t in medawar or t in notmedawar)]
all_df["Tissue type"] = all_df["Tissue"].apply(lambda t: "Medawar" if t in medawar else "Not Medawar")
all_df["Medawar"] = all_df["Tissue"].apply(lambda t: t in medawar)
all_df[r'$\beta_{age}$'] = all_df["Age Coef"]
for i in range(len(pathways)):
    pathway = list(pathways.keys())[i]
    print(pathway)
    ls = all_df[all_df["gene"].apply(lambda g: g in pathways[pathway])]
    sns.displot(data=ls, x=r'$\beta_{age}$', hue="Tissue type", kind="kde", fill=True, palette=[red,blue], common_norm=False, aspect=2, alpha=0.5)
    plt.show()
    plt.clf()
    print(scipy.stats.mannwhitneyu(ls[ls["Medawar"]]["Age Coef"], ls[~ls["Medawar"]]["Age Coef"]))

# pLI vs $h^2$

In [ ]:
# Plot pLI vs genetic R^2
import math
tissues = glob(gtex_dir+"GTEx_Analysis_v8_eQTL_expression_matrices/*.bed.gz")
all_data = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/all_res2.csv")
#short_list = ["Colon_Sigmoid", "Lung"]
#tissues = [t for t in tissues if np.any([s in t for s in short_list])]
print(tissues)
#plidf = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/exac_data_pLI.txt", sep="\t")
#plidf = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/gnomad_constraint_metrics.tsv", sep="\t")
plidf = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/gnomad.v2.1.1.lof_metrics.by_gene.txt", sep="\t")
meta_res = []
for i in range(len(tissues)):
    print(int(subprocess.check_output("zcat %s | awk '{print NF; exit}'" % tissues[i], shell=True)))
    tiss = tissues[i].split("expression_matrices/")[1].split(".v8.normalized_expression")[0]

    if int(subprocess.check_output("zcat %s | awk '{print NF; exit}'" % tissues[i], shell=True)) > 200 and exists(data_dir+"output/res_%s_agereg_proc0.npy" % tiss): # checks if num of individuals is enough
        print("Loaded res %s" % tiss)
        gene_r2 = output["gene_r2"]

        # save data as csv
        df = all_data[all_data["Tissue"]==tiss]
        print("Loaded res %s" % tiss)
        subdf = df[df["Genetic R^2"]>0]
        subdf["Genetic R^2"] = subdf["Genetic R^2"].astype(float)
    
        #thresh = 0.1
        #subdf["|Age R^2| > %s" % thresh] = subdf["|Young-Old R^2|"] > thresh

        import matplotlib
        matplotlib.rcParams["figure.dpi"] = 400
        fig, ax = plt.subplots()
        subdf["$h^2$"] = subdf["Genetic R^2"]
        res = dv.regplot(data=subdf, y="$h^2$", x="pLI", order=1, ax=ax,
                         scatter_kws={'s':3, 'alpha':0.2}, line_kws={'c':"black", 'linewidth':3})
        plt.text(0.5, max(list(subdf["$h^2$"]))-0.05, 'Fit slope: %.4f; P-value: %.1e' % (res.params[1], res.pvalues[1]) if res.pvalues[1]!=0 else 'fit slope: %.4f; P-value: 0' % (res.params[1]), size=14, ha="center")
        plt.xlabel("pLI", fontsize = 16)
        plt.ylabel("$h^2$", fontsize = 16)
        #sns.regplot(data=subdf, y="|Young-Old R^2|", x="pLI")
        plt.title("pLI vs $h^2$ for %s" % (re.sub('[_]', ' ', tiss)), fontsize = 18)
        plt.show()
        plt.clf()
        thresh = 0.5


        y = "Genetic R^2"
        x = "pLI > %s" % thresh
        #subdf["|Young-Old R^2| > %s" % thresh] = subdf["|Young-Old R^2| > %s" % thresh].astype(str)
        subdf["pLI > %s" % thresh] = (subdf["pLI"]>thresh).astype(str)
        ax = sns.violinplot(x=x, y=y, data=subdf, order=["True","False"])
        test_results = add_stat_annotation(ax, data=subdf, x=x, y=y,
                                           test='Mann-Whitney', text_format='full', loc='inside',
                                           box_pairs=[["True","False"]]) 
        Means = subdf.groupby(x)[y].mean()
        print(Means)
        plt.scatter(x=range(len(Means)),y=[Means["True"], Means["False"]],c="r")
        plt.title("%s for %s" % (y, tiss))
        plt.figure(dpi=400)
        plt.show()
        plt.clf()

        ax = sns.violinplot(x=x, y=y, data=subdf, order=["True","False"])
        plt.title("%s for %s; %s, %s" % (y, tiss, len(subdf[subdf[x]=="True"]), len(subdf[subdf[x]=="False"])))
        Q1 = np.percentile(sorted(list(subdf[y])), 25, interpolation = 'midpoint')
        Q80 = np.percentile(sorted(list(subdf[y])), 80, interpolation = 'midpoint')
        Means = subdf.groupby(x)[y].mean()
        plt.scatter(x=range(len(Means)),y=[Means["True"], Means["False"]],c="r")
        mean_diff = Means["True"] - Means["False"]
        ax.set_ylim([Q1, Q80])
        #ax.set(ylim=(0, 100000))
        plt.show()
        plt.clf()
        meta_res += [[tiss, res.params[1], res.pvalues[1], len(subdf), mean_diff, list(subdf["Genetic R^2"])]]
        #print(meta_res[-1])

In [ ]:
# Plot genetic r2 vs pli slope for each tissue
plt.rcParams["figure.dpi"] = 400
meta_res_dict = {r[0]:[r[1:]] for r in meta_res}
df = pd.DataFrame(np.array(meta_res)[:,:-1], columns=["Tissue", "Slope of Genetic R^2 vs pLI", "P-value", "Num Genes", "Mean Coef Diff"])
print(df)
df["Slope of Genetic R^2vs pLI"] = df["Slope of Genetic R^2 vs pLI"].astype(float)
df = df.sort_values("Slope of Genetic R^2 vs pLI")
df["Mean Coef Diff"] = df["Mean Coef Diff"].astype(float)
print(df)
df["Slope of $h^2$ vs pLI"] = df["Slope of Genetic R^2 vs pLI"]
df = df.merge(colors, on="Tissue")
df["Tissue"] = df["Tissue"].apply(lambda x: re.sub('[_]', ' ', x))
ax=sns.barplot(data=df, x="Slope of $h^2$ vs pLI", y="Tissue", palette=list(df["pal.tissue"]))
print(df)

def pval_asterisk(pvalue):
    if pvalue <= 0.0001:
        return "****"
    elif pvalue <= 0.001:
        return "***"
    elif pvalue <= 0.01:
        return "**"
    elif pvalue <= 0.05:
        return "*"
    return "ns"

plt.yticks(fontsize=8)
plt.xticks(fontsize=8)

x_position = df["Slope of Genetic R^2 vs pLI"] + np.sign(df["Slope of Genetic R^2 vs pLI"])*0.002
for idx, pval in enumerate(df["P-value"].apply(pval_asterisk)):
    plt.text(x=x_position.iloc[idx], y=(idx+0.3 if pval=="ns" else idx+0.6), s=pval, ha="center", size=9)
plt.text(x=-0.06, y=21.5, s="*: P-value<0.05\n**: P-value<0.01\n***: P-value<0.001\n****: P-value<0.0001", va="top", size=9)
plt.title("Relationship between $h^2$ and pLI")
plt.xlim(np.min(df["Slope of Genetic R^2 vs pLI"])-0.01, 0)
plt.show()
plt.clf()

# Facet plot of enriched genes

In [ ]:
# Facet plot of enriched genes
# Plot pLI vs age coef
from matplotlib.lines import Line2D
plidf = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/gnomad_constraint_metrics.tsv", sep="\t")
meta_res = []
cons_metric = "pLI"
quad = False # show quadrant colors and proportions
quadrants = []
import matplotlib
matplotlib.rcParams["figure.dpi"] = 200
f, big_axes = plt.subplots(figsize=(20.0, 8.0) , nrows=2, ncols=1, sharey=True, sharex=True) 
for row, big_ax in enumerate(big_axes, start=1):
    if row == 1:
        big_ax.set_title("Medawar\n", fontsize=16)
    else:
        big_ax.set_title("Non-Medawar\n", fontsize=16)

    big_ax.tick_params(labelcolor=(1.,1.,1., 0.0), top='off', bottom='off', left='off', right='off')
    big_ax._frameon = False
f.add_subplot(111, frameon=False)
plt.xlabel(cons_metric, fontsize=18)
plt.ylabel(r'$\beta_{Age}$', fontsize=18)
plt.title(r'%s vs $\beta_{Age}$ with Enriched Pathways' % cons_metric + "\n", fontsize=24)

f.set_facecolor('w')
plt.tight_layout()
tissues = ["Artery_Aorta", "Artery_Tibial", "Colon_Sigmoid", "Esophagus_Gastroesophageal_Junction", "Esophagus_Muscularis", "Breast_Mammary_Tissue", "Colon_Transverse", "Lung", "Prostate", "Whole_Blood"]
for i in range(len(tissues)):
    ax = f.add_subplot(2,5,i+1)
    tiss = tissues[i]
    all_data = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/all_res2.csv")
    
    subdf = df[df["Age R^2"]>0]
    subdf = subdf.drop_duplicates(subset=["Gene"])
    subdf["Age Coef"] = subdf["Age Coef"].astype(float)
    subdf["|Age Coef|"] = np.abs(subdf["Age Coef"])
    subdf = subdf.dropna()
    ax.set_title(re.sub('[_]', ' ', tiss))

    colors = ["orange", "black"]
    subdf["driving_colors"] = list(subdf["gene"].apply(lambda g: colors[int(g in driving_set)]))
    subdf = subdf.sort_values("driving_colors", ascending=False)
    res = dv.regplot(data=subdf, y="Age Coef", x=cons_metric, order=1, ax=ax,
                     scatter_kws={'s':3, 'alpha':0.3, 'c':subdf["driving_colors"]}, line_kws={'c':"black", 'linewidth':0})
    if i==4:
        legend_elements = [Line2D([0], [0], marker='o', color='black', label='GSEA pathway genes', markersize=5, linewidth=0),
                       Line2D([0], [0], marker='o', color='orange', label='Other genes', markersize=5, linewidth=0)]
        ax.legend(handles=legend_elements, loc='upper right')

    if cons_metric == "oe_lof":
        plt.xlim(0, 2)

plt.show()
plt.clf()

# Environment R^2

In [ ]:
# Plot pLI vs env R^2
import math
tissues = glob(gtex_dir+"GTEx_Analysis_v8_eQTL_expression_matrices/*.bed.gz")
plidf = pd.read_csv("/clusterfs/nilah/rkchung/data/expred/gnomad_constraint_metrics.tsv", sep="\t")
meta_res = []
for i in range(len(tissues)):
    print(int(subprocess.check_output("zcat %s | awk '{print NF; exit}'" % tissues[i], shell=True)))
    tiss = tissues[i].split("expression_matrices/")[1].split(".v8.normalized_expression")[0]

    if int(subprocess.check_output("zcat %s | awk '{print NF; exit}'" % tissues[i], shell=True)) > 200 and exists(data_dir+"output/res_%s_agereg_proc0.npy" % tiss): # checks if num of individuals is enough
        output = np.load(data_dir+"output/res_%s_agereg_proc0.npy"  % tiss, allow_pickle=True).item()
        print("Loaded res %s" % tiss)

        expr = pd.read_csv(gtex_dir+"GTEx_Analysis_v8_eQTL_expression_matrices/%s.v8.normalized_expression.bed.gz" % tiss, sep="\t")
        genes = expr["gene_id"].to_numpy()

        gene_r2 = output["gene_r2"]
        age_r2 = output["age_r2"]
        env_r2 = np.minimum(np.ones(len(gene_r2)) - gene_r2 - age_r2, 1)
        print(env_r2)

        # save data as csv
        df = pd.DataFrame(np.array([genes[:len(env_r2)], np.array(env_r2).reshape(-1)]).transpose(), columns=["Gene", "Env R^2"])
        df["gene"] = df["Gene"].apply(lambda x: mapping[x.split(".")[0]] if x.split(".")[0] in mapping else np.nan)
        df = df[["Gene", "gene", "Env R^2"]]
        print(df)
        df = df.merge(plidf[["gene", "pLI"]], on="gene")

        subdf = df
        subdf = subdf.drop_duplicates(subset=["Gene"])
        #subdf = df
        subdf["Env R^2"] = subdf["Env R^2"].astype(float)
        print(subdf)
    
        #thresh = 0.1
        #subdf["|Age R^2| > %s" % thresh] = subdf["|Young-Old R^2|"] > thresh

        import matplotlib
        matplotlib.rcParams["figure.dpi"] = 400
        fig, ax = plt.subplots()
        res = dv.regplot(data=subdf, y="Env R^2", x="pLI", order=1, ax=ax, scatter_kws={'s':2, 'alpha':0.1}, line_kws={'c':"black"})
        plt.text(0.2, max(0.5, 'Fit slope: %.4f; P-value: %.1e' % (res.params[1], res.pvalues[1]) if res.pvalues[1]!=0 else 'fit slope: %.4f; P-value: 0' % (res.params[1]))
        plt.xlabel("pLI")
        plt.ylabel("Env R^2")
        #sns.regplot(data=subdf, y="|Young-Old R^2|", x="pLI")
        plt.title("pLI vs Environmental R^2 for %s" % (tiss))
        plt.show()
        plt.clf()
        thresh = 0.5


        y = "Env R^2"
        x = "pLI > %s" % thresh
        #subdf["|Young-Old R^2| > %s" % thresh] = subdf["|Young-Old R^2| > %s" % thresh].astype(str)
        subdf["pLI > %s" % thresh] = (subdf["pLI"]>thresh).astype(str)
        ax = sns.violinplot(x=x, y=y, data=subdf, order=["True","False"])
        test_results = add_stat_annotation(ax, data=subdf, x=x, y=y,
                                           test='Mann-Whitney', text_format='full', loc='inside',
                                           box_pairs=[["True","False"]]) 
        Means = subdf.groupby(x)[y].mean()
        print(Means)
        plt.scatter(x=range(len(Means)),y=[Means["True"], Means["False"]],c="r")
        plt.title("%s for %s" % (y, tiss))
        plt.figure(dpi=400)
        plt.show()
        plt.clf()

        ax = sns.violinplot(x=x, y=y, data=subdf, order=["True","False"])
        plt.title("%s for %s; %s, %s" % (y, tiss, len(subdf[subdf[x]=="True"]), len(subdf[subdf[x]=="False"])))
        Q1 = np.percentile(sorted(list(subdf[y])), 25, interpolation = 'midpoint')
        Q80 = np.percentile(sorted(list(subdf[y])), 80, interpolation = 'midpoint')
        Means = subdf.groupby(x)[y].mean()
        plt.scatter(x=range(len(Means)),y=[Means["True"], Means["False"]],c="r")
        mean_diff = Means["True"] - Means["False"]
        ax.set_ylim([Q1, Q80])
        #ax.set(ylim=(0, 100000))
        plt.show()
        plt.clf()
        meta_res += [[tiss, res.params[1], res.pvalues[1], len(subdf), mean_diff, list(subdf["Env R^2"])]]
        #print(meta_res[-1])

In [ ]:
# Plot env r2 vs pli slope for each tissue
save = True
if save:
    np.save("envr2pli_results",np.array(meta_res))
else:
    meta_res = np.load("envr2pli_results.npy", use_pickle=True)
meta_res_dict = {r[0]:[r[1:]] for r in meta_res}
#ordered_res = [[t]+meta_res_dict[t][0][:-1] for t in tiss_order]
df = pd.DataFrame(np.array(meta_res)[:,:-1], columns=["Tissue", "Slope of Env R^2 vs pLI", "P-value", "Num Genes", "Mean Coef Diff"])
df["Slope of Env R^2 vs pLI"] = df["Slope of Env R^2 vs pLI"].astype(float)
df = df.sort_values("Slope of Env R^2 vs pLI")
df["Mean Coef Diff"] = df["Mean Coef Diff"].astype(float)
print(df)
ax=sns.barplot(data=df, x="Slope of Env R^2 vs pLI", y="Tissue")

def pval_asterisk(pvalue):
    if pvalue <= 0.0001:
        return "****"
    elif pvalue <= 0.001:
        return "***"
    elif pvalue <= 0.01:
        return "**"
    elif pvalue <= 0.05:
        return "*"
    return "ns"

plt.yticks(fontsize=8)
x_position = df["Slope of Env R^2 vs pLI"] + np.sign(df["Slope of Env R^2 vs pLI"])*0.003
for idx, pval in enumerate(df["P-value"].apply(pval_asterisk)):
    plt.text(x=x_position.iloc[idx], y=(idx+0.3 if pval=="ns" else idx+0.6), s=pval, ha="center")
plt.text(x=0.01, y=0, s="*: P-value<0.05\n**: P-value<0.01\n***: P-value<0.001\n****: P-value<0.0001", va="top")
plt.title("Relationship between Env R^2 and pLI")
plt.xlim(np.min(df["Slope of Env R^2 vs pLI"])-0.012, np.max(df["Slope of Env R^2 vs pLI"])+0.012)
plt.show()
plt.clf()